# 3. Feature Engineering

Este notebook implementa a criação de novas features para o dataset de doenças cardíacas.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from sklearn.preprocessing import StandardScaler, LabelEncoder

# Configurações de visualização
plt.style.use('seaborn')
sns.set_palette('husl')
pd.set_option('display.max_columns', None)

In [ ]:
# Carregar dados
data_path = Path('../data/HeartDiseaseTrain-Test.csv')
df = pd.read_csv(data_path)

## 1. Features de Risco por Idade

In [ ]:
# Criar categorias de risco por idade
df['age_risk'] = pd.cut(df['age'], 
                        bins=[0, 40, 50, 60, 70, 100], 
                        labels=[1, 2, 3, 4, 5])
df['age_risk'] = df['age_risk'].astype(int)

# Visualizar distribuição
plt.figure(figsize=(10, 6))
sns.countplot(data=df, x='age_risk', hue='target')
plt.title('Distribuição de Risco por Idade')
plt.show()

## 2. Features de Risco por Pressão Arterial

In [ ]:
# Criar categorias de risco por pressão arterial
def categorize_bp(bp):
    if bp < 120: return 1  # Normal
    elif bp < 130: return 2  # Elevated
    elif bp < 140: return 3  # Stage 1
    else: return 4  # Stage 2

df['bp_risk'] = df['resting_blood_pressure'].apply(categorize_bp)

# Visualizar distribuição
plt.figure(figsize=(10, 6))
sns.countplot(data=df, x='bp_risk', hue='target')
plt.title('Distribuição de Risco por Pressão Arterial')
plt.show()

## 3. Índice de Estresse Cardíaco

In [ ]:
# Calcular índice de estresse cardíaco
max_theoretical_heart_rate = 220 - df['age'].astype(float)
df['cardiac_stress_index'] = (
    df['Max_heart_rate'].astype(float) * 
    df['resting_blood_pressure'].astype(float) / 
    max_theoretical_heart_rate
)

# Visualizar distribuição
plt.figure(figsize=(10, 6))
sns.histplot(data=df, x='cardiac_stress_index', hue='target')
plt.title('Distribuição do Índice de Estresse Cardíaco')
plt.show()

## 4. Score de Risco Combinado

In [ ]:
# Calcular score de risco combinado
fasting_sugar_risk = (df['fasting_blood_sugar'] == 'Greater than 120 mg/ml').astype(int)
df['combined_risk_score'] = (
    df['age_risk'] +
    df['bp_risk'] +
    (df['cholestoral'] > 200).astype(int) * 2 +
    fasting_sugar_risk * 2
)

# Visualizar distribuição
plt.figure(figsize=(10, 6))
sns.boxplot(data=df, x='target', y='combined_risk_score')
plt.title('Score de Risco Combinado por Grupo')
plt.show()

## 5. Features de Interação

In [ ]:
# Criar features de interação
df['age_heartrate_interaction'] = df['age'].astype(float) * df['Max_heart_rate'].astype(float)
df['bp_cholesterol_interaction'] = df['resting_blood_pressure'].astype(float) * df['cholestoral'].astype(float) / 1000

# Visualizar correlações
interaction_features = ['age_heartrate_interaction', 'bp_cholesterol_interaction', 'target']
plt.figure(figsize=(8, 6))
sns.heatmap(df[interaction_features].corr(), annot=True, cmap='coolwarm')
plt.title('Correlação das Features de Interação')
plt.show()

## 6. Salvar Dataset Processado

In [ ]:
# Salvar dataset com novas features
output_path = Path('../data/heart_disease_processed.csv')
df.to_csv(output_path, index=False)
print(f"Dataset processado salvo em: {output_path}")